# Knowledge Graph Essentials

We'll explore our CRM graph and see how data is connected.

In [ ]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

load_dotenv()
driver = GraphDatabase.driver(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))
)

## Explore Customer Orders

In [ ]:
def get_customer_orders(customer_id):
    with driver.session() as session:
        result = session.run("""
        MATCH (c:Customer {customer_id: $customer_id})
        MATCH path = (c)-[:PLACED_ORDER]->(o:Order)-[:CONTAINS]->(p:Product)
        RETURN c.name as customer, 
               collect(p.name) as products,
               count(o) as order_count
        """, customer_id=customer_id)
        
        return result.single()

# Try it with customer C1
print(get_customer_orders('C1'))

## Find Related Products

In [ ]:
def find_related_products(product_id):
    with driver.session() as session:
        result = session.run("""
        MATCH (p1:Product {product_id: $product_id})-[:IN_CATEGORY]->(cat:Category)
        MATCH (p2:Product)-[:IN_CATEGORY]->(cat)
        WHERE p1 <> p2
        RETURN p2.name as related_product,
               cat.name as category
        """, product_id=product_id)
        
        return list(result)

# Find products related to P1 (Laptop Pro)
print(find_related_products('P1'))

## Customer Purchase Patterns

In [ ]:
def analyze_purchase_patterns():
    with driver.session() as session:
        result = session.run("""
        MATCH (cat:Category)<-[:IN_CATEGORY]-(p:Product)
                           <-[:CONTAINS]-(o:Order)
                           <-[:PLACED_ORDER]-(c:Customer)
        RETURN cat.name as category,
               count(DISTINCT c) as customer_count,
               count(o) as order_count
        ORDER BY order_count DESC
        """)
        
        return list(result)

print("Purchase Patterns by Category:")
for record in analyze_purchase_patterns():
    print(f"{record['category']}: {record['order_count']} orders by {record['customer_count']} customers")

## Next Steps

Now that we understand our graph structure, let's see how to process and connect documents!